In [60]:
import pandas as pd
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
import numpy as np

In [84]:
test_df = pd.read_csv('data/20190527_test.csv')
train_df = pd.read_csv('data/train.csv')
train_df.dropna(inplace=True)
X = train_df['review'].values
Y = train_df['label'].values
Y = [1 if x == 'Positive' else -1 for x in Y]

train_x, test_x, train_y, test_y = train_test_split(X,Y,test_size=0.2)

In [85]:
tfidf_model = TfidfVectorizer(ngram_range=(1,1),min_df=1).fit(train_x)
train_vec = tfidf_model.transform(train_x)
test_vec = tfidf_model.transform(test_x)

In [87]:
for i in range(1,8):
    print(i,end=':')
    lr_model = LogisticRegression(C=i)
    lr_model.fit(train_vec, train_y)
    res = lr_model.predict(test_vec)
    print(roc_auc_score(test_y, res))
rfc_model = RandomForestClassifier().fit(train_vec, train_y)
res1 = rfc_model.predict(test_vec)
print(roc_auc_score(test_y, res))

nb_model = BernoulliNB().fit(train_vec, train_y)
res2 = nb_model.predict(test_vec)
print(roc_auc_score(test_y, res2))




1:0.7475206611570248
2:0.7595041322314049
3:0.7675619834710745
4:0.7643939393939394
5:0.7594352617079889
6:0.7570247933884297
7:0.759228650137741


/home/dardis/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/dardis/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.759228650137741
0.768595041322314


In [67]:
np.concatenate((res1,res2),axis=0)

array([-1,  1,  1, ...,  1, -1,  1])

In [69]:
r1 = np.reshape(res1,(633,1))
r2 = np.reshape(res2,(633,1))
r = np.reshape(res,(633,1))

In [73]:
c = np.concatenate((r,r1,r2),axis=1)

In [74]:
c.shape

(633, 3)

In [79]:
layer2_model = LogisticRegression(C = 3).fit(c,test_y)

/home/dardis/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [80]:
rrr = layer2_model.predict(c)
print(roc_auc_score(test_y, rrr))

0.7670801869830995
